In [1]:
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os 
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics

In [2]:
All_Dropped_NA = pd.read_csv(r"Data\GTD_Styled_Data.csv", encoding = 'latin1', low_memory = False)
Grouped_Data_Set_Dropped_NA = pd.read_csv(r"C:\Users\scott\Documents\Graduate-Center-Class-Folder\QMSS Captstone\Terrorism-Capstone-Models\Data\Formatted_Year_Data.csv")

### Model 1: Type of Attack

#### Best Model : Region + PR + CL + Years


In [59]:
new_data = All_Dropped_NA[['iyear', 'CL', 'PR', 'region', 'attacktype1']].dropna()
x = new_data.drop('attacktype1', axis = 1)
y = new_data['attacktype1']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3,random_state=42, stratify=y)
x_train_encoded = pd.get_dummies(x_train, columns = ['region'])
x_test_encoded = pd.get_dummies(x_train, columns = ['region'])
logit_model = sm.MNLogit(y_train, x_train_encoded)
results = logit_model.fit_regularized(method = 'l1')
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2299: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2300: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2331: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2332: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)


Iteration limit reached    (Exit mode 9)
            Current function value: 1.3944732202023356
            Iterations: 1000
            Function evaluations: 1045
            Gradient evaluations: 1000


C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                          MNLogit Regression Results                          
Dep. Variable:            attacktype1   No. Observations:               121452
Model:                        MNLogit   Df Residuals:                   121332
Method:                           MLE   Df Model:                          112
Date:                Tue, 03 May 2022   Pseudo R-squ.:                 0.05049
Time:                        14:56:23   Log-Likelihood:            -1.6936e+05
converged:                      False   LL-Null:                   -1.7837e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
attacktype1=2       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
iyear             0.0169      0.001     15.056      0.000       0.015       0.019
CL               -0.0209      0.019     -1.120      0.263      -0.058       0.016
PR                0.0120      0.014     

In [67]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test_encoded)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train_encoded)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test_encoded.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.198761650693278

Test Accuracy:  0.19816721739798662

Test - No. Of Correct Predictions 10315.0 / 52052

Test Confusion Matrix:
[[  819  4454     0     0     0     3     0     0     0]
 [ 2735  9493     0     0     0    39     0     0     0]
 [ 2132 23180     0     0     0    53     0     0     0]
 [   34   139     0     0     0     0     0     0     0]
 [   83   205     0     0     0     4     0     0     0]
 [  497  2707     0     0     0     3     0     0     0]
 [  354  2532     0     0     0    89     0     0     0]
 [   19   244     0     0     0    14     0     0     0]
 [  398  1821     0     0     0     1     0     0     0]]

Test Precision = 0.062603
Test Recall = 0.198167
Test F1 Score = 0.091996

Classification Report:
              precision    recall  f1-score   support

           1       0.12      0.16      0.13      5276
           2       0.21      0.77      0.33     12267
           3       0.00      0.00      0.00     25365
           4       0.0

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott

### 2nd Best Model : GDP + PR + CL + Years

In [3]:
new_data = All_Dropped_NA[['iyear', 'GDP', 'CL', 'PR', 'attacktype1']].dropna()
x = new_data.drop('attacktype1', axis = 1)
y = new_data['attacktype1']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3,random_state=42, stratify=y)
logit_model = sm.MNLogit(y_train, x_train)
results = logit_model.fit_regularized(method = 'l1')
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

Inequality constraints incompatible    (Exit mode 4)
            Current function value: 2.1972245773362236
            Iterations: 1
            Function evaluations: 1
            Gradient evaluations: 1


C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                          MNLogit Regression Results                          
Dep. Variable:            attacktype1   No. Observations:               121452
Model:                        MNLogit   Df Residuals:                   121420
Method:                           MLE   Df Model:                           24
Date:                Wed, 04 May 2022   Pseudo R-squ.:                 -0.4961
Time:                        22:23:05   Log-Likelihood:            -2.6686e+05
converged:                      False   LL-Null:                   -1.7837e+05
Covariance Type:            nonrobust   LLR p-value:                     1.000
attacktype1=2       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
iyear                  0   2.07e-05          0      1.000   -4.05e-05    4.05e-05
GDP                    0    1.1e-14          0      1.000   -2.16e-14    2.16e-14
CL                     0      0.018     

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))

#### Actual 2nd Best Model

In [3]:
new_data = All_Dropped_NA[['iyear', 'CL', 'PR','GDP', 'attacktype1']].dropna()
x = new_data.drop('attacktype1', axis = 1)
y = new_data['attacktype1']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state=42, stratify=y)
logit_model = sm.MNLogit(y_train, x_train)
results = logit_model.fit()
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

Optimization terminated successfully.
         Current function value: 1.436167
         Iterations 8
                          MNLogit Regression Results                          
Dep. Variable:            attacktype1   No. Observations:               121452
Model:                        MNLogit   Df Residuals:                   121420
Method:                           MLE   Df Model:                           24
Date:                Tue, 10 May 2022   Pseudo R-squ.:                 0.02210
Time:                        13:25:46   Log-Likelihood:            -1.7443e+05
converged:                       True   LL-Null:                   -1.7837e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
attacktype1=2       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
iyear            -0.0002   1.82e-05    -11.367      0.000      -0.000      -0.000
CL                0.

In [4]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.2347347100088924

Test Accuracy:  0.23489971566894643

Test - No. Of Correct Predictions 12227.0 / 52052

Test Confusion Matrix:
[[    0  5272     0     0     0     4     0     0     0]
 [    0 12226     0     0     0    41     0     0     0]
 [    0 25307     0     0     0    58     0     0     0]
 [    0   170     0     0     0     3     0     0     0]
 [    0   288     0     0     0     4     0     0     0]
 [    0  3206     0     0     0     1     0     0     0]
 [    0  2858     0     0     0   117     0     0     0]
 [    0   262     0     0     0    15     0     0     0]
 [    0  2220     0     0     0     0     0     0     0]]

Test Precision = 0.055867
Test Recall = 0.234900
Test F1 Score = 0.089969

Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00      5276
           2       0.24      1.00      0.38     12267
           3       0.00      0.00      0.00     25365
           4       0.

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott

#### 2nd Best Model : PR + CL + Years 

In [68]:
new_data = All_Dropped_NA[['iyear', 'CL', 'PR', 'attacktype1']].dropna()
x = new_data.drop('attacktype1', axis = 1)
y = new_data['attacktype1']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3,random_state=42, stratify=y)
logit_model = sm.MNLogit(y_train, x_train)
results = logit_model.fit()
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

Optimization terminated successfully.
         Current function value: 1.443315
         Iterations 8
                          MNLogit Regression Results                          
Dep. Variable:            attacktype1   No. Observations:               121452
Model:                        MNLogit   Df Residuals:                   121428
Method:                           MLE   Df Model:                           16
Date:                Tue, 03 May 2022   Pseudo R-squ.:                 0.01723
Time:                        15:09:36   Log-Likelihood:            -1.7529e+05
converged:                       True   LL-Null:                   -1.7837e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
attacktype1=2       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
iyear          6.336e-05   1.57e-05      4.036      0.000    3.26e-05    9.41e-05
CL                0.

In [73]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test_encoded.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.2356815861410269

Test Accuracy:  0.23566817797587028

Test - No. Of Correct Predictions 12267.0 / 52052

Test Confusion Matrix:
[[    0  5276     0     0     0     0     0     0     0]
 [    0 12267     0     0     0     0     0     0     0]
 [    0 25365     0     0     0     0     0     0     0]
 [    0   173     0     0     0     0     0     0     0]
 [    0   292     0     0     0     0     0     0     0]
 [    0  3207     0     0     0     0     0     0     0]
 [    0  2975     0     0     0     0     0     0     0]
 [    0   277     0     0     0     0     0     0     0]
 [    0  2220     0     0     0     0     0     0     0]]

Test Precision = 0.055539
Test Recall = 0.235668
Test F1 Score = 0.089894

Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00      5276
           2       0.24      1.00      0.38     12267
           3       0.00      0.00      0.00     25365
           4       0.

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott

#### 3rd Best Model : CL + Years

In [76]:
new_data = All_Dropped_NA[['iyear', 'CL', 'attacktype1']].dropna()
x = new_data.drop('attacktype1', axis = 1)
y = new_data['attacktype1']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state=42, stratify=y)
logit_model = sm.MNLogit(y_train, x_train)
results = logit_model.fit()
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

Optimization terminated successfully.
         Current function value: 1.443927
         Iterations 8
                          MNLogit Regression Results                          
Dep. Variable:            attacktype1   No. Observations:               121452
Model:                        MNLogit   Df Residuals:                   121436
Method:                           MLE   Df Model:                            8
Date:                Tue, 03 May 2022   Pseudo R-squ.:                 0.01681
Time:                        15:14:37   Log-Likelihood:            -1.7537e+05
converged:                       True   LL-Null:                   -1.7837e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
attacktype1=2       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
iyear          5.388e-05   1.53e-05      3.517      0.000    2.38e-05    8.39e-05
CL                0.

In [77]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test_encoded.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.2356815861410269

Test Accuracy:  0.23566817797587028

Test - No. Of Correct Predictions 12267.0 / 52052

Test Confusion Matrix:
[[    0  5276     0     0     0     0     0     0     0]
 [    0 12267     0     0     0     0     0     0     0]
 [    0 25365     0     0     0     0     0     0     0]
 [    0   173     0     0     0     0     0     0     0]
 [    0   292     0     0     0     0     0     0     0]
 [    0  3207     0     0     0     0     0     0     0]
 [    0  2975     0     0     0     0     0     0     0]
 [    0   277     0     0     0     0     0     0     0]
 [    0  2220     0     0     0     0     0     0     0]]

Test Precision = 0.055539
Test Recall = 0.235668
Test F1 Score = 0.089894

Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00      5276
           2       0.24      1.00      0.38     12267
           3       0.00      0.00      0.00     25365
           4       0.

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott

### Model 2: GDP

##### Best Model: Number of Attacks + Number of Deaths + Total Rating + Years

In [78]:
new_data = Grouped_Data_Set_Dropped_NA[['GDP','Number of Attacks', 'Number of Deaths', 'Total Rating', 'Years']]
x = new_data.drop('GDP', axis = 1)
y = new_data['GDP']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state=42)

data = pd.concat([y_train, x_train], axis = 1)
logit_model = sm.OLS(y_train, x_train)
result=logit_model.fit( maxiter=5000)
stats1=result.summary()
stats2=result.summary2()
print(stats1)
print(stats2)

                                 OLS Regression Results                                
Dep. Variable:                    GDP   R-squared (uncentered):                   0.117
Model:                            OLS   Adj. R-squared (uncentered):              0.115
Method:                 Least Squares   F-statistic:                              73.53
Date:                Tue, 03 May 2022   Prob (F-statistic):                    1.39e-58
Time:                        15:22:27   Log-Likelihood:                         -65389.
No. Observations:                2224   AIC:                                  1.308e+05
Df Residuals:                    2220   BIC:                                  1.308e+05
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [88]:
prediction = result.predict(x_test)
print("Mean Absolute Error", metrics.mean_absolute_error(y_test, prediction))
print("Mean Squared Error" , metrics.mean_squared_error(y_test, prediction))
print("Root Mean Squared Error", np.sqrt(metrics.mean_squared_error(y_test, prediction)))

Mean Absolute Error 588707968366.8252
Mean Squared Error 2.4902994011286247e+24
Root Mean Squared Error 1578068249832.2512


##### Interesting Model: Number of Deaths + Number of Attacks + PR + Years

In [89]:
new_data = Grouped_Data_Set_Dropped_NA[['GDP','Number of Attacks', 'Number of Deaths', 'PR', 'Years']]
x = new_data.drop('GDP', axis = 1)
y = new_data['GDP']


x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state=42)
logit_model = sm.OLS(y_train, x_train)
result=logit_model.fit( maxiter=5000)
stats1=result.summary()
stats2=result.summary2()
print(stats1)
print(stats2)

                                 OLS Regression Results                                
Dep. Variable:                    GDP   R-squared (uncentered):                   0.110
Model:                            OLS   Adj. R-squared (uncentered):              0.109
Method:                 Least Squares   F-statistic:                              68.80
Date:                Tue, 03 May 2022   Prob (F-statistic):                    5.71e-55
Time:                        15:35:40   Log-Likelihood:                         -65397.
No. Observations:                2224   AIC:                                  1.308e+05
Df Residuals:                    2220   BIC:                                  1.308e+05
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [90]:
prediction = result.predict(x_test)
print("Mean Absolute Error", metrics.mean_absolute_error(y_test, prediction))
print("Mean Squared Error" , metrics.mean_squared_error(y_test, prediction))
print("Root Mean Squared Error", np.sqrt(metrics.mean_squared_error(y_test, prediction)))

Mean Absolute Error 587587527244.6533
Mean Squared Error 2.50824835741822e+24
Root Mean Squared Error 1583745041797.5173


##### Interesting Model: Number of Deaths + Number of Attacks + CL + Years

In [91]:
new_data = Grouped_Data_Set_Dropped_NA[['GDP','Number of Attacks', 'Number of Deaths', 'CL', 'Years']]
x = new_data.drop('GDP', axis = 1)
y = new_data['GDP']


x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state=42)
logit_model = sm.OLS(y_train, x_train)
result=logit_model.fit( maxiter=5000)
stats1=result.summary()
stats2=result.summary2()
print(stats1)
print(stats2)

                                 OLS Regression Results                                
Dep. Variable:                    GDP   R-squared (uncentered):                   0.122
Model:                            OLS   Adj. R-squared (uncentered):              0.120
Method:                 Least Squares   F-statistic:                              76.81
Date:                Tue, 03 May 2022   Prob (F-statistic):                    4.44e-61
Time:                        15:36:27   Log-Likelihood:                         -65383.
No. Observations:                2224   AIC:                                  1.308e+05
Df Residuals:                    2220   BIC:                                  1.308e+05
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [92]:
prediction = result.predict(x_test)
print("Mean Absolute Error", metrics.mean_absolute_error(y_test, prediction))
print("Mean Squared Error" , metrics.mean_squared_error(y_test, prediction))
print("Root Mean Squared Error", np.sqrt(metrics.mean_squared_error(y_test, prediction)))

Mean Absolute Error 592887513869.2258
Mean Squared Error 2.473981789337668e+24
Root Mean Squared Error 1572889630373.876


### Model 3: Target of Attack

##### Best Model : Attack Type + Year


In [3]:
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
new_data = All_Dropped_NA[['iyear', 'attacktype1', 'targtype1']].dropna()
x = new_data.drop('targtype1', axis = 1)
y = new_data['targtype1']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 42, stratify=y)
x_train_encoded = pd.get_dummies(x_train, columns = ['attacktype1'])
x_test_encoded = pd.get_dummies(x_test, columns = ['attacktype1'])
logit_model = sm.MNLogit(y_train, x_train_encoded)
results = logit_model.fit_regularized(method = 'l1')
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2299: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2300: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2331: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2332: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)


Iteration limit reached    (Exit mode 9)
            Current function value: 2.1629227871964427
            Iterations: 1000
            Function evaluations: 1109
            Gradient evaluations: 1000


C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                          MNLogit Regression Results                          
Dep. Variable:              targtype1   No. Observations:               121452
Model:                        MNLogit   Df Residuals:                   121242
Method:                           MLE   Df Model:                          189
Date:                Wed, 04 May 2022   Pseudo R-squ.:                 0.06625
Time:                        17:59:57   Log-Likelihood:            -2.6269e+05
converged:                      False   LL-Null:                   -2.8133e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
  targtype1=2       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
iyear             0.0023      0.001      2.527      0.011       0.001       0.004
attacktype1_1    -2.8610      1.852     -1.545      0.122      -6.491       0.769
attacktype1_2    -4.2884      1.858     

In [4]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test_encoded)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train_encoded)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test_encoded.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.02966604090504891

Test Accuracy:  0.029259202336125414

Test - No. Of Correct Predictions 1523.0 / 52052

Test Confusion Matrix:
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0]
 [1646  238    0  468    0   18    0    0    0   26    0    0    0 3295
     0    0    0    0    0    0  172    0    0]
 [ 796  977    0  369    0    5    0    0    0   11    0    0    0 3835
     0    0    0    0    0    0   51    0    0]
 [ 420  613    0 1250    0    6    0    0    0    3    0    0    0 5016
     0    0    0    0    0    0   32    0    0]
 [ 302  287    0 2157    0    4    0    0    0    2    0    0    0 5070
     0    0    0    0    0    0   41    0    0]
 [  63    3    0    2    0    0    0    0    0    0    0    0    0    8
     0    0    0    0    0    0    0    0    0]
 [  94    1    0   26    0   61    0    0    0    3    0    0    0  128
     0    0    0    0    0    0   25    0    0]
 [ 203   6

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data 

##### 2nd Best Model: Number of Killed + Attack Type + Year 


In [102]:

new_data = All_Dropped_NA[['iyear', 'nkill', 'attacktype1', 'targtype1']].dropna()
x = new_data.drop('targtype1', axis = 1)
y = new_data['targtype1']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 42, stratify=y)
x_train_encoded = pd.get_dummies(x_train, columns = ['attacktype1'])
x_test_encoded = pd.get_dummies(x_test, columns = ['attacktype1'])
logit_model = sm.MNLogit(y_train, x_train_encoded)
results = logit_model.fit_regularized(method = 'l1')
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2299: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2300: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2331: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2332: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2663014136476654
            Iterations: 70
            Function evaluations: 354
            Gradient evaluations: 70
                          MNLogit Regression Results                          
Dep. Variable:              targtype1   No. Observations:               114622
Model:                        MNLogit   Df Residuals:                   114391
Method:                           MLE   Df Model:                          210
Date:                Tue, 03 May 2022   Pseudo R-squ.:                 0.02235
Time:                        17:28:20   Log-Likelihood:            -2.5977e+05
converged:                       True   LL-Null:                   -2.6571e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
  targtype1=2       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test_encoded)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train_encoded)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test_encoded.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.02966604090504891

Test Accuracy:  0.029259202336125414

Test - No. Of Correct Predictions 1523.0 / 52052

Test Confusion Matrix:
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0]
 [1646  238    0  468    0   18    0    0    0   26    0    0    0 3295
     0    0    0    0    0    0  172    0    0]
 [ 796  977    0  369    0    5    0    0    0   11    0    0    0 3835
     0    0    0    0    0    0   51    0    0]
 [ 420  613    0 1250    0    6    0    0    0    3    0    0    0 5016
     0    0    0    0    0    0   32    0    0]
 [ 302  287    0 2157    0    4    0    0    0    2    0    0    0 5070
     0    0    0    0    0    0   41    0    0]
 [  63    3    0    2    0    0    0    0    0    0    0    0    0    8
     0    0    0    0    0    0    0    0    0]
 [  94    1    0   26    0   61    0    0    0    3    0    0    0  128
     0    0    0    0    0    0   25    0    0]
 [ 203   6

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data 

##### 3rd Best Model : Number of Killed + Year


In [6]:
new_data = All_Dropped_NA[['iyear', 'nkill', 'targtype1']].dropna()
x = new_data.drop('targtype1', axis = 1)
y = new_data['targtype1']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 42, stratify=y)

logit_model = sm.MNLogit(y_train, x_train)
results = logit_model.fit_regularized(method = 'l1')
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2299: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2300: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2331: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2332: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.291543775258335
            Iterations: 71
            Function evaluations: 347
            Gradient evaluations: 71
                          MNLogit Regression Results                          
Dep. Variable:              targtype1   No. Observations:               114622
Model:                        MNLogit   Df Residuals:                   114580
Method:                           MLE   Df Model:                           21
Date:                Wed, 04 May 2022   Pseudo R-squ.:                 0.01147
Time:                        18:13:49   Log-Likelihood:            -2.6266e+05
converged:                       True   LL-Null:                   -2.6571e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
 targtype1=2       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.0008200868943134826

Test Accuracy:  0.0008142493638676844

Test - No. Of Correct Predictions 40.0 / 49125

Test Confusion Matrix:
[[    0     0     1     0     0     0     0     0     0     0     0     0
   5443     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
   5788     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
   7118     0     0     0     0     0     0     0     0     0]
 [    0     0     2     0     0     0     0     0     0     0     0     0
   7175     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
     76     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
    312     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0  

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott

### Model 4: Region 

##### Best Model: Type of Attack + Target of Attack + Years

In [8]:
new_data = All_Dropped_NA[['iyear', 'region', 'attacktype1', 'targtype1']].dropna()
x = new_data.drop('region', axis = 1)
y = new_data['region']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 42, stratify=y)
x_train_encoded = pd.get_dummies(x_train, columns = ['attacktype1', 'targtype1'])
x_test_encoded = pd.get_dummies(x_test, columns = ['attacktype1', 'targtype1'])
logit_model = sm.MNLogit(y_train, x_train_encoded)
results = logit_model.fit_regularized(method = 'l1')
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2299: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2300: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2331: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2332: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)


Iteration limit reached    (Exit mode 9)
            Current function value: 1.6195515190505905
            Iterations: 1000
            Function evaluations: 1060
            Gradient evaluations: 1000


C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:4247: RuntimeWarning: invalid value encountered in sqrt
  bse = np.sqrt(np.diag(self.cov_params()))


                          MNLogit Regression Results                          
Dep. Variable:                 region   No. Observations:               121452
Model:                        MNLogit   Df Residuals:                   121100
Method:                           MLE   Df Model:                          341
Date:                Wed, 04 May 2022   Pseudo R-squ.:                  0.1670
Time:                        18:25:53   Log-Likelihood:            -1.9670e+05
converged:                      False   LL-Null:                   -2.3613e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
     region=2       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
iyear            -0.0665      0.002    -29.329      0.000      -0.071      -0.062
attacktype1_1    93.2593        nan        nan        nan         nan         nan
attacktype1_2    93.7845        nan     

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test_encoded)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train_encoded)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test_encoded.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.08179363040542766

Test Accuracy:  0.08270575578267886

Test - No. Of Correct Predictions 4305.0 / 52052

Test Confusion Matrix:
[[   0    0    0    0    0    0    0    0    0    0    0    0    0]
 [  75   12  182    0    0  191    0  312    0   55   22    0    0]
 [   4  821 1310    0    3  103    0  579    0   67   49    0    0]
 [   3  320 2131    1    7  824    0 1508    0  457  109    0    0]
 [   1    0   31    0    0   64    0   64    0   34    7    0    0]
 [   1  109  281    0    9 2265    0  232    0  943   58    0    0]
 [   5  118  559    0    6 8416    0  501    0 4353  240    0    0]
 [   0    7   16    0    0   73    0    9    0   47    6    0    0]
 [   4  206  838    0    4  722    0 2223    0  624   75    0    0]
 [   3    6   27    0    0  715    0   41    0  529   24    0    0]
 [   3  184  404    0    7 5016    0  432    0 7184  179    0    0]
 [   1   36  338    0    4 2964    0  271    0 1085  230    0    0]
 [   0    2   13    0    2   29    

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data 

##### 2nd Model: Type of Attack + Years

In [10]:
new_data = All_Dropped_NA[['iyear', 'region', 'attacktype1']].dropna()
x = new_data.drop('region', axis = 1)
y = new_data['region']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 42, stratify=y)
x_train_encoded = pd.get_dummies(x_train, columns = ['attacktype1'])
x_test_encoded = pd.get_dummies(x_test, columns = ['attacktype1'])
logit_model = sm.MNLogit(y_train, x_train_encoded)
results = logit_model.fit_regularized(method = 'l1')
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2299: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2300: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2331: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2332: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)


Iteration limit reached    (Exit mode 9)
            Current function value: 1.6521552973021512
            Iterations: 1000
            Function evaluations: 1080
            Gradient evaluations: 1000


C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                          MNLogit Regression Results                          
Dep. Variable:                 region   No. Observations:               121452
Model:                        MNLogit   Df Residuals:                   121342
Method:                           MLE   Df Model:                           99
Date:                Wed, 04 May 2022   Pseudo R-squ.:                  0.1502
Time:                        18:30:32   Log-Likelihood:            -2.0066e+05
converged:                      False   LL-Null:                   -2.3613e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
     region=2       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
iyear            -0.0910      0.002    -40.213      0.000      -0.095      -0.087
attacktype1_1   182.5688      4.509     40.487      0.000     173.731     191.407
attacktype1_2   183.7793      4.522     

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test_encoded)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train_encoded)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test_encoded.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.07290946217435695

Test Accuracy:  0.07200491815876431

Test - No. Of Correct Predictions 3748.0 / 52052

Test Confusion Matrix:
[[  42  116    0    0  238    0  383    0   66    4    0    0]
 [1234  951    0    0  142    0  543    0   62    4    0    0]
 [ 814 2300    0    0  755    0  856    0  628    7    0    0]
 [  13   27    0    0   43    0   53    0   60    5    0    0]
 [ 137  284    0    0 2007    0  166    0 1286   18    0    0]
 [ 352  494    0    0 6687    0  386    0 6251   28    0    0]
 [   7    4    0    0   71    0    4    0   71    1    0    0]
 [ 324 1065    0    0  616    0 1792    0  889   10    0    0]
 [   3    8    0    0  581    0   28    0  719    6    0    0]
 [ 286  310    0    0 4130    0  375    0 8283   25    0    0]
 [ 135  339    0    0 3012    0  168    0 1245   30    0    0]
 [   6   11    0    0   30    0   15    0   10    1    0    0]]

Test Precision = 0.064774
Test Recall = 0.072005
Test F1 Score = 0.028722

Classification Rep

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott

##### 3rd Model: Target of Attack + Years

In [12]:
new_data = All_Dropped_NA[['iyear', 'region', 'targtype1']].dropna()
x = new_data.drop('region', axis = 1)
y = new_data['region']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 42, stratify=y)
x_train_encoded = pd.get_dummies(x_train, columns = ['targtype1'])
x_test_encoded = pd.get_dummies(x_test, columns = ['targtype1'])
logit_model = sm.MNLogit(y_train, x_train_encoded)
results = logit_model.fit_regularized(method = 'l1')
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2299: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2300: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2331: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2332: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)


Iteration limit reached    (Exit mode 9)
            Current function value: 1.6925458096875519
            Iterations: 1000
            Function evaluations: 1060
            Gradient evaluations: 1000


C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                          MNLogit Regression Results                          
Dep. Variable:                 region   No. Observations:               121452
Model:                        MNLogit   Df Residuals:                   121199
Method:                           MLE   Df Model:                          242
Date:                Wed, 04 May 2022   Pseudo R-squ.:                  0.1294
Time:                        18:38:12   Log-Likelihood:            -2.0556e+05
converged:                      False   LL-Null:                   -2.3613e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
    region=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
iyear           -0.0572      0.002    -26.590      0.000      -0.061      -0.053
targtype1_1    114.4984      4.297     26.648      0.000     106.077     122.920
targtype1_2    115.5229      4.302     26.85

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test_encoded)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train_encoded)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test_encoded.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.06247735730988374

Test Accuracy:  0.062091754399446704

Test - No. Of Correct Predictions 3232.0 / 52052

Test Confusion Matrix:
[[   0    0    0    0    0    0    0    0    0    0    0    0    0]
 [  74   22  161    0    0  140    0  280    0  170    2    0    0]
 [   0  865 1036    0    8  117    0  801    0   90   19    0    0]
 [   0  391 1958    0    8  760    0 1597    0  634   12    0    0]
 [   0    4   53    0    1   71    0   35    0   36    1    0    0]
 [   0  117  241    0   23 1485    0  210    0 1812   10    0    0]
 [   0  150  541    0   13 7060    0  410    0 6004   20    0    0]
 [   0    3   19    0    0   74    0    4    0   52    6    0    0]
 [   0  320 1066    0    5  704    0 1794    2  772   33    0    0]
 [   0    1   36    0    0  601    0   36    0  654   17    0    0]
 [   0  213  367    0    8 4545    0  444    0 7804   28    0    0]
 [   0   35  354    0   17 1386    0  232    0 2882   23    0    0]
 [   0    1   10    0    2   27   

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data 

### Model 5: Government Total Score

##### Best Model: Type of Attack + Target of Attack + Years

In [14]:
new_data = All_Dropped_NA[['iyear', 'Total Rating', 'attacktype1', 'targtype1']].dropna()
x = new_data.drop('Total Rating', axis = 1)
y = new_data['Total Rating']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 42, stratify=y)
x_train_encoded = pd.get_dummies(x_train, columns = ['attacktype1', 'targtype1'])
x_test_encoded = pd.get_dummies(x_test, columns = ['attacktype1', 'targtype1'])
logit_model = sm.MNLogit(y_train, x_train_encoded)
results = logit_model.fit_regularized(method = 'l1')
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2299: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2300: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2331: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2332: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)


Iteration limit reached    (Exit mode 9)
            Current function value: 2.182736385703541
            Iterations: 1000
            Function evaluations: 1066
            Gradient evaluations: 1000


C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:4247: RuntimeWarning: invalid value encountered in sqrt
  bse = np.sqrt(np.diag(self.cov_params()))


                          MNLogit Regression Results                          
Dep. Variable:           Total Rating   No. Observations:               121452
Model:                        MNLogit   Df Residuals:                   121068
Method:                           MLE   Df Model:                          372
Date:                Wed, 04 May 2022   Pseudo R-squ.:                 0.07683
Time:                        18:55:51   Log-Likelihood:            -2.6510e+05
converged:                      False   LL-Null:                   -2.8716e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
 Total Rating=3       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
iyear              -0.0352      0.001    -24.274      0.000      -0.038      -0.032
attacktype1_1      47.2076   2.27e+05      0.000      1.000   -4.45e+05    4.45e+05
attacktype1_2      47.9552   3.0

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test_encoded)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train_encoded)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test_encoded.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.0896732865658861

Test Accuracy:  0.08971797433335894

Test - No. Of Correct Predictions 4670.0 / 52052

Test Confusion Matrix:
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [ 409  395    4 1101   66   24    1   80    2  521   57    0   14    0
     0]
 [ 249  445    3 1045   53   15    0   83    1  572   52    0    6    0
     0]
 [  43  171    1  553   23   17    0   68    1  171   22    0    1    0
     0]
 [ 246  331    3 2136  208  159    5  597    2 1771  626    0   10    0
     0]
 [ 183  192    0  969  196  144    6  435    0 1611  353    0   19    0
     0]
 [ 118  153    2  984  276  249    6  289    0  810  168    0   10    0
     0]
 [ 141   90    2  945   84   75    9  539    2 1399  321    0   19    0
     0]
 [ 146  109    0 1120  128   76    5 1339    1 2991  655    0   21    0
     0]
 [  43  129    2  786  123   23    0  175   12  483 

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data 

##### 2nd Model: Type of Attack + Years 

In [16]:
new_data = All_Dropped_NA[['iyear', 'Total Rating', 'attacktype1']].dropna()
x = new_data.drop('Total Rating', axis = 1)
y = new_data['Total Rating']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 42, stratify=y)
x_train_encoded = pd.get_dummies(x_train, columns = ['attacktype1'])
x_test_encoded = pd.get_dummies(x_test, columns = ['attacktype1'])
logit_model = sm.MNLogit(y_train, x_train_encoded)
results = logit_model.fit_regularized(method = 'l1')
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2299: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2300: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2331: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2332: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)


Iteration limit reached    (Exit mode 9)
            Current function value: 2.2078817721886126
            Iterations: 1000
            Function evaluations: 1066
            Gradient evaluations: 1000


C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                          MNLogit Regression Results                          
Dep. Variable:           Total Rating   No. Observations:               121452
Model:                        MNLogit   Df Residuals:                   121332
Method:                           MLE   Df Model:                          108
Date:                Wed, 04 May 2022   Pseudo R-squ.:                 0.06619
Time:                        19:01:22   Log-Likelihood:            -2.6815e+05
converged:                      False   LL-Null:                   -2.8716e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
 Total Rating=3       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
iyear              -0.0354      0.001    -25.445      0.000      -0.038      -0.033
attacktype1_1      69.9885      2.771     25.255      0.000      64.557      75.420
attacktype1_2      70.4393      

In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test_encoded)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train_encoded)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test_encoded.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.09389717748575568

Test Accuracy:  0.09298393913778528

Test - No. Of Correct Predictions 4840.0 / 52052

Test Confusion Matrix:
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [ 358  606    0 1027    0    0    0   55   42  568   14    0    4    0
     0]
 [ 314  378    0 1037    0    0    0   58    6  726    4    0    1    0
     0]
 [  43  291    0  490    0    0    0   28    5  183   31    0    0    0
     0]
 [ 333  495    0 2273    0    0    0  590    5 1882  509    0    7    0
     0]
 [ 258   42    0 1406    0    0    0  241    5 1695  450    0   11    0
     0]
 [ 153   61    0 1577    0    0    0  214    5  903  146    0    6    0
     0]
 [ 201   64    0 1037    0    0    0  437    2 1623  252    0   10    0
     0]
 [ 208  204    0 1218    0    0    0  807    1 3512  634    0    7    0
     0]
 [  65  231    0  823    0    0    0  172   13  536

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data 

##### 3rd Model: Target Of Attack + Years 


In [18]:
new_data = All_Dropped_NA[['iyear', 'Total Rating', 'targtype1']].dropna()
x = new_data.drop('Total Rating', axis = 1)
y = new_data['Total Rating']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 42, stratify=y)
x_train_encoded = pd.get_dummies(x_train, columns = ['targtype1'])
x_test_encoded = pd.get_dummies(x_test, columns = ['targtype1'])
logit_model = sm.MNLogit(y_train, x_train_encoded)
results = logit_model.fit_regularized(method = 'l1')
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2299: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2300: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2331: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2332: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)


Iteration limit reached    (Exit mode 9)
            Current function value: 2.245769500709168
            Iterations: 1000
            Function evaluations: 1067
            Gradient evaluations: 1000


C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                          MNLogit Regression Results                          
Dep. Variable:           Total Rating   No. Observations:               121452
Model:                        MNLogit   Df Residuals:                   121176
Method:                           MLE   Df Model:                          264
Date:                Wed, 04 May 2022   Pseudo R-squ.:                 0.05017
Time:                        19:10:37   Log-Likelihood:            -2.7275e+05
converged:                      False   LL-Null:                   -2.8716e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
 Total Rating=3       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
iyear              -0.0309      0.001    -21.873      0.000      -0.034      -0.028
targtype1_1        62.0139      2.825     21.953      0.000      56.477      67.550
targtype1_2        62.4363      

In [19]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test_encoded)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train_encoded)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test_encoded.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.12431248559101538

Test Accuracy:  0.12391454699147007

Test - No. Of Correct Predictions 6450.0 / 52052

Test Confusion Matrix:
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [ 122  119    0 1440   39   20    2   70    0  810   52    0    0    0
     0]
 [  62   79    0 1635   74   20    2   67    0  546   39    0    0    0
     0]
 [  19   30    0  718   19   16    0   50    0  205   14    0    0    0
     0]
 [  64   53    0 2381  108  160    7  272    0 2887  162    0    0    0
     0]
 [  39   12    0 1176  205  144   10  129    0 2276  117    0    0    0
     0]
 [  33   16    0 1195  239  250    7  172    0 1069   84    0    0    0
     0]
 [  43   13    0 1099   81   84   11  249    0 1946  100    0    0    0
     0]
 [  39   19    0 1260  137   76    9  554    0 4292  205    0    0    0
     0]
 [  17   30    0  944   82   24    9   94    0  542

C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\scott\anaconda3\envs\Data 

### Model 6: Gov Type using Other Data Set

In [14]:
new_data = Grouped_Data_Set_Dropped_NA[['Number of Attacks', 'Total Rating', 'Years']].dropna()
x = new_data.drop('Total Rating', axis = 1)
y = new_data['Total Rating']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.5, random_state = 42, stratify=y)

logit_model = sm.MNLogit(y_train, x_train)
results = logit_model.fit()
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

Optimization terminated successfully.
         Current function value: 2.478459
         Iterations 7
                          MNLogit Regression Results                          
Dep. Variable:           Total Rating   No. Observations:                 1589
Model:                        MNLogit   Df Residuals:                     1565
Method:                           MLE   Df Model:                           12
Date:                Sun, 15 May 2022   Pseudo R-squ.:                 0.01016
Time:                        00:25:01   Log-Likelihood:                -3938.3
converged:                       True   LL-Null:                       -3978.7
Covariance Type:            nonrobust   LLR p-value:                 2.870e-12
   Total Rating=3       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Number of Attacks     0.0045      0.002      2.494      0.013       0.001       0.008
Years   

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.010698552548772814

Test Accuracy:  0.010069225928256766

Test - No. Of Correct Predictions 16.0 / 1589

Test Confusion Matrix:
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [269   0   0   0   0   0   0   0   5   0   0   0   0   0]
 [111   0   0   0   0   0   0   0  11   0   0   0   0   0]
 [ 91   0   0   0   0   0   0   0   3   0   0   0   0   0]
 [100   0   0   0   0   0   0   0  24   0   0   0   0   0]
 [ 98   0   0   0   0   0   0   0  12   0   0   0   0   0]
 [105   0   0   0   0   0   0   0  14   0   0   0   0   0]
 [ 92   0   0   0   0   0   0   0  15   0   0   0   0   0]
 [ 93   0   0   0   0   0   0   0  16   0   0   0   0   0]
 [105   0   0   0   0   0   0   0   7   0   0   0   0   0]
 [125   0   0   0   0   0   0   0  24   0   0   0   0   0]
 [ 97   0   0   0   0   0   0   0  11   0   0   0   0   0]
 [ 71   0   0   0   0   0   0   0   2   0   0   0   0   0]
 [ 78   0   0   0   0   0   0   0  10   0   0   0   0   0]]

Test Precision = 0.00712

c:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\scott\anaconda3\envs\Data 

In [16]:
new_data = Grouped_Data_Set_Dropped_NA[['Number of Deaths', 'Total Rating', 'Years']].dropna()
x = new_data.drop('Total Rating', axis = 1)
y = new_data['Total Rating']
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.5, stratify=y)

logit_model = sm.MNLogit(y_train, x_train)
results = logit_model.fit_regularized(method = 'l1')
results = logit_model.fit()
stats1 = results.summary()
stats2 = results.summary2()
print(stats1)
print(stats2)

c:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2299: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
c:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2300: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]
c:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2331: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
c:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\discrete\discrete_model.py:2332: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)
c:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Ma

Singular matrix E in LSQ subproblem    (Exit mode 5)
            Current function value: nan
            Iterations: 2
            Function evaluations: 12
            Gradient evaluations: 2
Optimization terminated successfully.
         Current function value: 2.464928
         Iterations 8
                          MNLogit Regression Results                          
Dep. Variable:           Total Rating   No. Observations:                 1589
Model:                        MNLogit   Df Residuals:                     1565
Method:                           MLE   Df Model:                           12
Date:                Sun, 15 May 2022   Pseudo R-squ.:                 0.01565
Time:                        00:25:24   Log-Likelihood:                -3916.8
converged:                       True   LL-Null:                       -3979.1
Covariance Type:            nonrobust   LLR p-value:                 7.549e-21
  Total Rating=3       coef    std err          z      P>|z|      [0.025  

In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
prediction_array = results.predict(x_test)
prediction = np.array(prediction_array).argmax(1)
training_array = results.predict(x_train)
training_prediction = np.array(training_array).argmax(1)
training_accuracy = np.mean(training_prediction == y_train)
test_accuracy = np.mean(prediction == y_test)

print("\nTrain Accuracy: ", training_accuracy)
print("\nTest Accuracy: ", test_accuracy)

test_size = x_test.shape[0]
test_no_correct = test_size  * test_accuracy

print ("\nTest - No. Of Correct Predictions", test_no_correct, "/", test_size)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, prediction))


precision_test = precision_score(y_test, prediction,average='weighted') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, prediction,average='weighted')
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, prediction,average='weighted')
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_test, prediction))


Train Accuracy:  0.015733165512901194

Test Accuracy:  0.010069225928256766

Test - No. Of Correct Predictions 16.0 / 1589

Test Confusion Matrix:
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [262   0   0   0   0   0   0   0  12   0   0   0   0   0]
 [109   0   0   0   0   0   0   0  14   0   0   0   0   0]
 [ 88   0   0   0   0   0   0   0   6   0   0   0   0   0]
 [ 98   0   0   0   0   0   2   0  24   0   0   0   0   0]
 [ 94   0   0   0   0   0   0   0  16   0   0   0   0   0]
 [104   0   0   0   0   0   0   0  14   0   0   0   0   0]
 [ 86   0   0   0   0   0   0   0  20   0   0   0   0   0]
 [ 93   0   0   0   0   0   0   0  16   0   0   0   0   0]
 [102   0   0   0   0   0   0   0  10   0   0   0   0   0]
 [122   0   0   0   0   0   4   0  24   0   0   0   0   0]
 [ 87   0   0   0   0   0   4   0  17   0   0   0   0   0]
 [ 65   0   0   0   0   0   0   0   8   0   0   0   0   0]
 [ 76   0   0   0   0   0   0   0  12   0   0   0   0   0]]

Test Precision = 0.00568

c:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\scott\anaconda3\envs\Data Science\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\scott\anaconda3\envs\Data 